In [12]:
import cv2
import numpy as np
import os

In [13]:
image = cv2.imread('GCT-NY.jpg', cv2.IMREAD_GRAYSCALE)
template_Face = cv2.imread('Face.jpg', cv2.IMREAD_GRAYSCALE)
template_GCT = cv2.imread('GCT-NY-GC.jpg', cv2.IMREAD_GRAYSCALE)
template_norm_Face = (template_Face - np.mean(template_Face)) / np.std(template_Face)
template_norm_GCT = (template_GCT - np.mean(template_GCT)) / np.std(template_GCT)

font = cv2.FONT_HERSHEY_SIMPLEX
org = (0,13)
fontScale = .3
color_thickness = ((0,2),(255,1)) # ((color,thickness),(color_out,thickness_out))

def mouse_callback(event, x, y, flags, param):
    global cursor_x, cursor_y
    if event == cv2.EVENT_LBUTTONDOWN:
        cursor_x, cursor_y = x, y
    elif event == cv2.EVENT_RBUTTONDOWN:
        i = 0
        name = 'capture.png'
        while os.path.exists(name):
            i += 1
            name = f'capture_{i}.png'
        cv2.imwrite(name, param)

In [14]:
def adjust_brightness(image, v):
    return cv2.add(image, v)

def adjust_contrast(image, v):
    return cv2.addWeighted(image, 1+(v/10), -np.mean(image), v/10, 0)

def hist(image):
    h = np.histogram(image.ravel(), 256, (0,255))[0]
    h = (h / max(1, h.max()) * 100).astype(int)
    img = ((np.arange(100)[:, None] >= (100 - h)) * 255).astype(np.uint8)
    return cv2.resize(img, (img.shape[1], 100), interpolation=cv2.INTER_NEAREST)

def windowing(frame, conf):
    counts, edges = np.histogram(frame.ravel(), bins=256, range=(0, 255))
    cdf = np.cumsum(counts) / counts.sum()
    l = int(edges[np.searchsorted(cdf, (1 - conf)/2)])
    h = int(edges[np.searchsorted(cdf, (conf+1)/2)])
    r = h-l
    return cv2.addWeighted(frame, 255/r, -l/r, 255, 0)

# def windowing_slow(frame, conf):
#     hist = np.zeros(256)
#     for row in frame:
#         for intensity in row:
#             hist[intensity] += 1
#     normalized_cdf = np.cumsum(hist) / image.size
#     l = np.argwhere(normalized_cdf >= (1-conf)/2)[0, 0]
#     h = np.argwhere(normalized_cdf >= (conf+1)/2)[0, 0]
#     r = h-l
#     return cv2.addWeighted(frame, 255/r, -l/r, 255, 0)

def equalize_hist(frame):
    counts, _ = np.histogram(frame.ravel(), bins=256, range=(0, 255))
    cdf = np.cumsum(counts)
    mapping = np.floor(cdf * 255 / frame.size)
    return mapping[frame.ravel()].astype(np.uint8).reshape(frame.shape)

# def equalize_hist_slow(I, levels):
#     hist = np.zeros(levels)
#     for row in I:
#         for intensity in row:
#             hist[intensity] += 1
#     cdf = np.zeros_like(hist)
#     cdf[0] = hist[0]
#     for i in range(1, len(hist)):
#         cdf[i] = cdf[i-1] + hist[i]
#     mapping = np.floor(cdf * 255 / I.size)
#     return mapping[I.ravel()].astype(np.uint8).reshape(I.shape)

def template_match(frame, template_norm):
    frame_norm = (frame - np.mean(frame)) / np.std(frame)
    corr_map = cv2.filter2D(frame_norm, -1, template_norm)
    y, x = np.unravel_index(np.argmax(corr_map), corr_map.shape)
    h, w = template_norm.shape[:2]
    cv2.rectangle(frame, (x-w//2, y-h//2), (x+w//2, y+h//2), 0, 2)
    cv2.rectangle(frame, (x-w//2, y-h//2), (x+w//2, y+h//2), 255, 1)
    cv2.circle(frame, (x, y), 5, 0, 2)
    cv2.circle(frame, (x, y), 3, 255, 1)

def add_gaussian_noise(frame, sigma):
    return cv2.addWeighted(frame, 1, np.random.randn(*frame.shape).astype(np.uint8), 255*sigma/10000, 0)
def add_salt_and_pepper_noise(frame, size):
    noise_index = np.random.choice(frame.size, size*frame.size//100) # p = size/100 for noise count
    noise = 255 * np.random.randint(0,2,noise_index.size)
    noisy_image = frame.flatten()
    noisy_image[noise_index] = noise
    return noisy_image.reshape(frame.shape)

def median_blur(frame, kernel_size):
    return cv2.medianBlur(frame, kernel_size)

def gaussian_blur(frame, kernel_size):
    return cv2.GaussianBlur(frame, (kernel_size, kernel_size), 0)

def average_blur(frame, kernel_size):
    return cv2.blur(frame, (kernel_size, kernel_size))

def bilateral_blur(frame, kernel_size, sigma_color, sigma_space):
    return cv2.bilateralFilter(frame, kernel_size, sigma_color, sigma_space)

def sharpen(frame, kernel_size):
    negative_kernel = np.full((kernel_size, kernel_size), -1/kernel_size**2)
    double_image = np.zeros_like(negative_kernel)
    double_image[kernel_size//2, kernel_size//2] = 2
    return cv2.filter2D(frame, -1, negative_kernel + double_image) # or 2*image-blur(image)

# def slow_otsu(frame):
#     uniques = np.unique(frame.flatten()).astype(np.uint16)
#     t, total_var = None, np.inf
#     for i in range(len(uniques) - 1):
#         sep = (uniques[i] + uniques[i+1])/2
#         l = uniques[uniques <= sep]
#         r = uniques[uniques > sep]
#         new_total_var=len(l) * np.var(l) + len(r) * np.var(r)
#         if new_total_var < total_var:
#             total_var = new_total_var
#             t = i
#     return np.where(frame > t, 0, 255).astype(np.uint8)

# def slow_adaptive_otsu(frame):
#     h,w = frame.shape
#     h_size = 30
#     w_size = 20
#     h_bins = h//h_size
#     w_bins = w//w_size
#     tiles = frame.reshape(h_bins, h_size, w_bins, w_size)
#     tiles = tiles.swapaxes(1, 2)
#     new_tiles = np.zeros_like(tiles)
#     for row_i in range(tiles.shape[0]):
#         for col_i in range(tiles.shape[1]):
#             new_tiles[row_i, col_i] = otsu(tiles[row_i, col_i])
#     return new_tiles.swapaxes(1,2).reshape(frame.shape).astype(np.uint8)

thresholds = ['off', 'mean', 'median', 'otsu', 'adaptive']
def threshold(frame, threshold_index):
    if threshold_index == 1:
        return np.where(frame > np.mean(frame), 0, 255).astype(np.uint8)
    elif threshold_index == 2:
        return np.where(frame > np.median(frame), 0, 255).astype(np.uint8)
    elif threshold_index == 3:
        return 255 - cv2.threshold(frame, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    elif threshold_index == 4:
        return 255 - cv2.adaptiveThreshold(frame, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 199, 10)

# from scipy.ndimage import generic_filter
# def dilation_slow(image, shape, size):
#     if shape == 'square':
#         kernel = np.ones(size**2)
#     else:
#         y, x = np.ogrid[:size, :size]
#         r = size//2
#         kernel = ((x-r)**2 + (y-r)**2 <= r**2).ravel()
#     return generic_filter(image,lambda image: bool(np.any(np.array(image)*kernel)),(size,size))

# def erosion_slow(image, shape, size):
#     if shape == 'square':
#         kernel = np.ones(size**2)
#     else:
#         y, x = np.ogrid[:size, :size]
#         r = size//2
#         kernel = ((x-r)**2 + (y-r)**2 <= r**2).ravel()
#     return generic_filter(image,lambda image: np.sum(np.array(image)*kernel) == np.sum(kernel),(size,size))

# opening_slow = lambda image, shape, size: dilation(erosion(image, shape, size), shape, size)
# closing_slow = lambda image, shape, size: erosion(dilation(image, shape, size), shape, size)



def extract_boundary(frame):
    return cv2.dilate(frame, np.ones((2,2))) - cv2.erode(frame, np.ones((2,2)))

def std_filter(img, ksize):
    return np.sqrt(cv2.boxFilter(img**2, -1, (ksize, ksize)) - cv2.boxFilter(img, -1, (ksize, ksize))**2
)
def zero_crossing(img, thrsh):
    img_shrx = img.copy()
    img_shrx[:, 1:] = img_shrx[:, :-1]
    img_shdy = img.copy()
    img_shdy[1:, :] = img_shdy[:-1, :]
    res = (img == 0) | (img * img_shrx < 0) | (img * img_shdy < 0)
    std_image = std_filter(img, 3) / img.max()
    res = res & (std_image > thrsh)
    return np.uint8(res)

# gx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
# gy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
# def canny_slow(frame, l, h):
#     gx_image = cv2.filter2D(frame, cv2.CV_32F, gx)
#     gy_image = cv2.filter2D(frame, cv2.CV_32F, gy)
#     g_magnitude = np.sqrt(gx_image**2 + gy_image**2)
#     g_angle = np.arctan2(gy_image, gx_image)
#     edge_map = np.where(g_magnitude >= h, g_magnitude, 0)
#     weak_edges = np.argwhere((h > g_magnitude) & (g_magnitude >= l))
#     for i, j in weak_edges:
#         if np.any(edge_map[i-1:i+2, j-1:j+2] == 255):
#             edge_map[i, j] = g_magnitude[i,j]
#         else:
#             edge_map[i, j] = 0
#     nms = np.zeros_like(edge_map, np.uint8)
#     rows, cols = edge_map.shape
#     for i in range(1, rows - 1):
#         for j in range(1, cols - 1):
#             angle = g_angle[i,j]
#             q = [0, 0]
#             if (-np.pi/8 <= angle < np.pi/8) or (7*np.pi/8 <= angle):
#                 q[0] = edge_map[i,j+1]
#                 q[1] = edge_map[i,j-1]
#             elif (np.pi/8 <= angle < 3*np.pi/8):
#                 q[0] = edge_map[i+1,j+1]
#                 q[1] = edge_map[i-1,j-1]
#             elif (3*np.pi/8 <= angle < 5*np.pi/8):
#                 q[0] = edge_map[i+1,j]
#                 q[1] = edge_map[i-1,j]
#             else:
#                 q[0] = edge_map[i-1,j+1]
#                 q[1] = edge_map[i+1,j-1]
            
#             if edge_map[i,j] > max(*q):
#                 nms[i,j] = 255
#     return nms

edge_detection_methods = ['off', 'sobel', 'canny', 'laplacian']
def edge_detection(frame, edge_detection_index, kernel_size,  canny_l, canny_h, laplacian_threshold):
    if edge_detection_index == 1:
        sobel_x = cv2.Sobel(frame, cv2.CV_64F, 1, 0, ksize=kernel_size)
        sobel_y = cv2.Sobel(frame, cv2.CV_64F, 0, 1, ksize=kernel_size)
        return np.sqrt(sobel_x**2 + sobel_y**2).astype(np.uint8)
    elif edge_detection_index == 2:
        return cv2.Canny(frame, canny_l, canny_h, apertureSize=kernel_size)
    elif edge_detection_index == 3:
        return (zero_crossing(cv2.Laplacian(
                frame, cv2.CV_64F, ksize=kernel_size), laplacian_threshold)).astype(np.uint8) * 255
    
# mode = ['off', 'line']
mode = ['off', 'line', 'circle']
# def hough_fit_slow(frame, hough_index, theta_bins, rho_bins, k, canny_l, canny_h, canny_kernel_size):
#     frame = cv2.Canny(frame, canny_l, canny_h, apertureSize=canny_kernel_size)
#     points = np.argwhere(frame == 255)
#     if hough_index == 1:
#         thetas = np.deg2rad(np.linspace(-90, 90, theta_bins), dtype=np.float32)
#         sc = np.array([np.sin(thetas), np.cos(thetas)], dtype=np.float32)
#         max_d = (frame.shape[0]**2+frame.shape[1]**2)**.5
#         rhos = np.linspace(-max_d, max_d, rho_bins + 1, dtype=np.float32)
#         rho_centers = (rhos[:-1] + rhos[1:])/2
#         rhos_per_thetas = (points @ sc).astype(np.float32).T
#         rhos_binned = np.digitize(rhos_per_thetas, rhos) - 1
#         hough = np.zeros((rho_bins,theta_bins), dtype=np.uint32)
#         for theta_idx in range(theta_bins):
#             np.add.at(hough[:, theta_idx], rhos_binned[theta_idx], 1)
#         hr, hc = np.unravel_index(np.argsort(hough, axis=None)[-k:], hough.shape)
#         hough = (hough/hough.max() * 255).astype(np.uint8)
#         for i in range(k):
#             y = lambda x: (rho_centers[hr[i]] - x * np.cos(thetas[hc[i]])) / np.sin(thetas[hc[i]])
#             x = np.array([0, frame.shape[1]])
#             try:
#                 cv2.line(frame, (int(x[0]), int(y(x[0]))), (int(x[1]), int(y(x[1]))), 255, 2)
#                 cv2.line(frame, (int(x[0]), int(y(x[0]))), (int(x[1]), int(y(x[1]))), 0, 1)
#             except Exception as e:
#                 print(f"Error drawing line: {e}")
#                 continue
#     elif hough_index == 2:
#         return cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)
#     return frame, hough


def create_hough_map(frame, edges, rho_res, deg_res, status='normal'):
    thetas = np.deg2rad(np.arange(-90, 90, deg_res), dtype=np.float32)
    max_d = (frame.shape[0]**2+frame.shape[1]**2)**.5
    rhos = np.arange(-max_d, max_d, rho_res + 1, dtype=np.float32)
    if status == 'fail':
        return np.zeros_like(frame, dtype=np.uint8)
    points = np.argwhere(edges == 255)
    sc = np.array([np.sin(thetas), np.cos(thetas)], dtype=np.float32)
    rhos_per_thetas = (points @ sc).astype(np.float32).T
    rhos_binned = np.digitize(rhos_per_thetas, rhos) - 1
    hough = np.zeros((len(rhos),len(thetas)), dtype=np.uint32)
    for theta_idx in range(len(thetas)):
        np.add.at(hough[:, theta_idx], rhos_binned[theta_idx], 1)
    hough = (hough/hough.max() * 255).astype(np.uint8)
    hough = cv2.resize(hough, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    return hough

def hough_fit(frame, hough_index, rho_res, deg_res, t, canny_l, canny_h):
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    if hough_index == 1:
        edges = cv2.Canny(blurred, canny_l, canny_h)
        lines = cv2.HoughLines(edges, rho_res, np.deg2rad(deg_res), t)
        if lines is None:
            return frame, create_hough_map(frame, edges, rho_res, deg_res, 'fail')
        for line in lines:
            rho,theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))        
            cv2.line(frame,(x1,y1),(x2,y2),255,2)
            cv2.line(frame,(x1,y1),(x2,y2),0,1)
        hough = create_hough_map(frame, edges, rho_res, deg_res)
        return frame, hough
    elif hough_index == 2:
        circles = cv2.HoughCircles(blurred,cv2.HOUGH_GRADIENT,1,rho_res, # In fact, it is minimum distance between centers. 
                            param1=canny_h,param2=20,minRadius=1, maxRadius=min(frame.shape)//2)
        if circles is None:
            return frame
        circles = np.uint16(circles)
        for x,y,r in circles[0,:]:
            cv2.circle(frame,(x,y),r,255,2)
            cv2.circle(frame,(x,y),r,0,1)
            # draw the center of the circle
            cv2.circle(frame,(x,y),2,255,2)
            cv2.circle(frame,(x,y),2,0,1)
        return frame
    
# def transform_slow(img, m):
#     indicies_tar = np.array(list(np.ndindex(img.shape)), np.uint16)
#     hindex = np.hstack([indicies_tar, np.ones((len(indicies_tar), 1),dtype=np.uint16)]).reshape(len(indicies_tar), 3, 1)
#     horg = np.float32(np.linalg.inv(m) @ hindex)
#     horg[:, :2, 0] /= horg[:, 2, [0]]
#     indicies_org = np.uint16(horg[:, :2, 0])
#     transformed_img = np.zeros_like(img, np.uint8)
#     for i, (y, x) in enumerate(indicies_tar):
#         yorg,xorg = indicies_org[i]
#         if 0 <= yorg < img.shape[0] and 0 <= xorg < img.shape[1]:
#             transformed_img[y, x] = img[*indicies_org[i]]
#     return transformed_img

def transform(frame, m):
    return cv2.warpPerspective(frame, m, frame.shape)

In [19]:
I = np.eye(3, dtype=np.float32)
cursor_x, cursor_y = 0, 0
cam = cv2.VideoCapture(0)
ms = int(1000/cam.get(cv2.CAP_PROP_FPS))
brightness = 0
contrast = 0
gaussian_noise = 0
salt_and_pepper_noise = 0
median_kernel = 1
gaussian_kernel = 1
average_kernel = 1
bilateral_kernel = 1
bilateral_sigma_color = 0
bilateral_sigma_space = 0
camera = True
hist_switch = True
windowing_switch = False
windowing_confidence = .8
hist_eq_switch = False
template_match_switch = False
sharpen_kernel = 1
threshold_index = 0
edge_detection_index = 0
edge_detection_kernel = 3
canny_l = 100
canny_h = 200
laplacian_threshold = .2
boundary_extraction = False
hough_index = 0
deg_res = 1
rho_res = 1
hough_t = 50
perspective_transformation = I.copy()

while True:
    if camera:
        frame = cv2.cvtColor(cam.read()[1], cv2.COLOR_RGB2GRAY)
        template = template_norm_Face
    else:
        frame = np.copy(image)
        template = template_norm_GCT
    if (perspective_transformation != I).any():
        frame = transform(frame, perspective_transformation)
    if brightness:
        frame = adjust_brightness(frame, brightness)
    if contrast:
        frame = adjust_contrast(frame, contrast)
    if gaussian_noise:
        frame = add_gaussian_noise(frame, gaussian_noise)
    if salt_and_pepper_noise:
        frame = add_salt_and_pepper_noise(frame, salt_and_pepper_noise)
    if windowing_switch:
        frame = windowing(frame, windowing_confidence)
    if hist_eq_switch:
        frame = equalize_hist(frame)
    if template_match_switch:
        template_match(frame, template_norm_Face if camera else template_norm_GCT)
    if median_kernel != 1:
        frame = median_blur(frame, median_kernel)
    if gaussian_kernel != 1:
        frame = gaussian_blur(frame, gaussian_kernel)
    if average_kernel != 1:
        frame = average_blur(frame, average_kernel)
    if bilateral_kernel != 1:
        frame = bilateral_blur(frame, bilateral_kernel, bilateral_sigma_color, bilateral_sigma_space)
    if sharpen_kernel != 1:
        frame = sharpen(frame, sharpen_kernel)
    if threshold_index:
        frame = threshold(frame, threshold_index)
    if boundary_extraction:
        frame = extract_boundary(frame)
    if edge_detection_index:
        frame = edge_detection(frame, edge_detection_index, edge_detection_kernel, canny_l, canny_h, laplacian_threshold)
    if hough_index == 1:
            frame, hough_map = hough_fit(frame, hough_index, rho_res, deg_res, hough_t, canny_l, canny_h)
            cv2.imshow('Hough Map', hough_map)
    elif hough_index == 2:
        try:
            cv2.destroyWindow('Hough Map')
        except:
            pass
        frame = hough_fit(frame, hough_index, rho_res, deg_res, hough_t, canny_l, canny_h)
    if hist_switch:
        frame_hist = hist(frame)
        cv2.imshow('Hist', frame_hist)
    else:
        try:
            cv2.destroyWindow('Hist')
        except:
            pass

    for (color, thickness) in color_thickness:
        cv2.putText(frame,
                    f'color={frame[cursor_y, cursor_x]}, Brightness={brightness}, Contrast={contrast}, sp noise={salt_and_pepper_noise}, gaussian noise={gaussian_noise}, Histogram={hist_switch}, Camera={camera}, Windowing={windowing_switch}',
                    org, font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame,
                    f'Windowing={windowing_switch}, Histogram Equalization={hist_eq_switch}, template_matching={template_match_switch},median_kernel = {median_kernel}, gaussian_kernel = {gaussian_kernel}, threshold={thresholds[threshold_index]}',
                    (0,2*org[1]), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame,
                    f'average_kernel = {average_kernel}, bilateral_kernel = {bilateral_kernel}, bilateral_sigma_color = {bilateral_sigma_color}, bilateral_sigma_space = {bilateral_sigma_space}, sharpen_kernel = {sharpen_kernel}',
                    (0,3*org[1]), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame,
                    f'edge_detection={edge_detection_methods[edge_detection_index]}, edge_detection_kernel={edge_detection_kernel}, canny_l_threshold={canny_l}, canny_h_threshold={canny_h}, laplacian_threshold={laplacian_threshold}',
                    (0,4*org[1]), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame,
                    f'boundary_extraction={boundary_extraction}, hough_fit={mode[hough_index]}, rho/center_res={rho_res}, deg_res={deg_res}, hough_vote/t={hough_t}, a={perspective_transformation[0,0]}, b={perspective_transformation[0,1]}, c={perspective_transformation[1,0]}, d={perspective_transformation[1,1]}, tx={perspective_transformation[0,2]}, d1={perspective_transformation[2,0]}, d2={perspective_transformation[2,1]}',
                    (0,5*org[1]), font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('Main', frame)
    cv2.setMouseCallback('Main', mouse_callback, param=frame)

    key = cv2.waitKey(ms) & 0xFF
    if key == 55 : # ord('7')
        perspective_transformation[0,0] -= .05
    if key == 38 : # ord('&')
        perspective_transformation[0,0] += .05
    if key == 56 : # ord('8')
        perspective_transformation[0,1] -= .05
    if key == 42 : # ord('*')
        perspective_transformation[0,1] += 10
    if key == 57 : # ord('9')
        perspective_transformation[0,2] -= 10
    if key == 40 : # ord('(')
        perspective_transformation[0,2] += .05
    if key == 52 : # ord('4')
        perspective_transformation[1,0] -= .05
    if key == 36 : # ord('$')
        perspective_transformation[1,0] += .05
    if key == 53 : # ord('5')
        perspective_transformation[1,1] -= .05
    if key == 37 : # ord('%')
        perspective_transformation[1,1] += .05
    if key == 54 : # ord('6')
        perspective_transformation[1,2] -= 10
    if key == 94 : # ord('^')
        perspective_transformation[1,2] += 10
    if key == 49 : # ord('1')
        perspective_transformation[2,0] -= .05
    if key == 33 : # ord('!')
        perspective_transformation[2,0] += .05
    if key == 50 : # ord('2')
        perspective_transformation[2,1] -= .001
    if key == 64 : # ord('@')
        perspective_transformation[2,1] += .001
    if key == 104: # ord('h')
        hist_switch = not hist_switch
    if key == 72: # ord('H')
        hough_index = (hough_index + 1) % len(mode)
    if key == 114: # ord('r')
        if rho_res > 1:
            rho_res -= 1
    if key == 82: # ord('R')
        rho_res += 1
    if key == 59: # ord(';')
        if deg_res > 1:
            deg_res -= 1
    if key == 58: # ord(':')
        deg_res += 1
    if key == 47: # ord('/')
        if hough_t > 1:
            hough_t -= 1
    if key == 63: # ord('?')
        hough_t += 1
    if key == 119: # ord('w')
        windowing_switch = not windowing_switch
    if key == 87: # ord('W')
        hist_eq_switch = not hist_eq_switch
    if key == 88: # ord('X')
        if contrast < 10:
            contrast += 1
    if key == 120: #  ord('x')
        if contrast > -10:
            contrast -= 1
    if key == 66: #  ord('B')
        if brightness < 255:
            brightness += 1
    if key == 98: #  ord('b')
        if brightness > -255:
            brightness -= 1
    if key == 103: # ord('g')
        if gaussian_noise > 0:
            gaussian_noise -= 1
    if key == 71: # ord('G')
        if gaussian_noise < 10:
            gaussian_noise += 1
    if key == 112: # ord('p')
        if salt_and_pepper_noise > 0:
            salt_and_pepper_noise -= 1
    if key == 80: # ord('P')
        if salt_and_pepper_noise < 10:
            salt_and_pepper_noise += 1
    if key == 109: # ord('m')
        if median_kernel > 1:
            median_kernel -= 2
    if key == 77: # ord('M'):
        median_kernel += 2
    if key == 122: # ord('z')
        if gaussian_kernel > 1:
            gaussian_kernel -= 2
    if key == 90: # ord('Z')
        gaussian_kernel += 2
    if key == 108: # ord('l')
        if bilateral_kernel > 1:
            bilateral_kernel -= 2
    if key == 76: # ord('L')
        bilateral_kernel += 2
    if key == 97: # ord('a')
        if average_kernel > 1:
            average_kernel -= 2
    if key == 65: # ord('A')
        average_kernel += 2
    if key == 43: # ord('+')
        if bilateral_sigma_color < 10:
            bilateral_sigma_color += 1
    if key == 61: # ord('=')
        if bilateral_sigma_color > 0:
            bilateral_sigma_color -= 1
    if key == 45: # ord('-')
        if bilateral_sigma_space > 0:
            bilateral_sigma_space -= 1
    if key == 95: # ord('_')
        if bilateral_sigma_space < 10:
            bilateral_sigma_space += 1
    if key == 115: # ord('s')
        if sharpen_kernel > 1:
            sharpen_kernel -= 2
    if key == 83: # ord('S')
        sharpen_kernel += 2
    if key == 124: # ord('|'):
        threshold_index = (threshold_index + 1) % len(thresholds)
    if key == 101: # ord('e')
        edge_detection_index = (edge_detection_index + 1) % len(edge_detection_methods)
    if key == 93: # ord(']')
        if edge_detection_kernel > 3:
            edge_detection_kernel -= 2
    if key == 125: # ord('}')
        if edge_detection_kernel < 7:
            edge_detection_kernel +=2
    if key == 46: # ord('.')
        if canny_h > 0:
            canny_h -= 1
    if key == 62: # ord('>')
        canny_h += 1
    if key == 44: # ord(',')
        if canny_l > 0:
            canny_l -= 1
    if key == 60: # ord('<')
        canny_l += 1
    if key == 39: # ord("'")
        if laplacian_threshold > 0:
            laplacian_threshold -= .05
    if key == 34: # ord('"')
        laplacian_threshold += .05
    if key == 99: # ord('c')
        camera = not camera
        cursor_x, cursor_y = 0, 0
        continue
    if key == 116: # ord('t')
        template_match_switch = not template_match_switch
    if key == 84: # ord('T'):
        # SIFT
        pass
    if key == 111: # ord('o')
        boundary_extraction = not boundary_extraction
    if key == 113: #  ord('q')
        break

cam.release()
cv2.destroyAllWindows()